The purpose of this notebook is for two things.

1. Explore and examine the TSV file containing the data of locations
2. Implementation of edit distance computation using dynamic programming

In [24]:
import csv
import math

In [25]:
# Incrase field size limit for this tsv file. Fixes some weird bug
csv.field_size_limit(1000000)

131072

In [212]:
# Covert tsv file to csv file
# Practice reading value from tsv and csv
# NOT NECESSARY
with open('cities_canada-usa.tsv','r',encoding='UTF-8') as tsvin, open('new.csv', 'w') as csvout:
    tsvin = csv.reader(tsvin, delimiter='\t')
    csvout = csv.writer(csvout) 
    for index, row in enumerate(tsvin):
        csvout.writerow(row)

In [233]:
def edge_d(a, b):
    '''
    Compute the edit distance between two words using dynamic programming
    Input: Word a, Word b
    Output: Graph of distances. A 2-dimensional array
    '''
    
    # Create graph nxm
    n = len(a)
    m = len(b)
    graph = []
    
    # Edge Case
    if n == 0:
        return m
    elif m == 0:
        return n
    
    # One extra column / row for initial empty state
    n += 1
    m += 1
    
    for i in range(n):
        graph.append([0] * m)
    
    # Call helper with graph
    return helper(a, b, graph)

def helper(a, b, graph):
    
    # Solve smallest subproblem 
    n = len(a) + 1
    m = len(b) + 1
    
    for i in range(n):
        graph[i][0] = i
    
    for i in range(1, m):
        graph[0][i] = i
        
    # Solve subproblems recursively
    for i in range(1, n):
        for j in range(1, m):
            diff = 1 if a[i-1] != b[j-1] else 0
            graph[i][j] = min(graph[i-1][j] + 1, graph[i][j-1] + 1, graph[i-1][j-1] + diff)
    return graph
    
edge_d('exponential', 'polynomial')

# edge_d('Calgary', '')
# edge_d('Snow', 'Know')

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [2, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [3, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10],
 [4, 3, 2, 3, 4, 5, 5, 6, 7, 8, 9],
 [5, 4, 3, 3, 4, 4, 5, 6, 7, 8, 9],
 [6, 5, 4, 4, 4, 5, 5, 6, 7, 8, 9],
 [7, 6, 5, 5, 5, 4, 5, 6, 7, 8, 9],
 [8, 7, 6, 6, 6, 5, 5, 6, 7, 8, 9],
 [9, 8, 7, 7, 7, 6, 6, 6, 6, 7, 8],
 [10, 9, 8, 8, 8, 7, 7, 7, 7, 6, 7],
 [11, 10, 9, 8, 9, 8, 8, 8, 8, 7, 6]]

## Create a table called cities

In [117]:
import sys
import psycopg2
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

create_table_command = (
    """
    CREATE TABLE cities (
        id TEXT PRIMARY KEY,
        name VARCHAR(200),
        ascii VARCHAR(200),
        alt_name VARCHAR(5000),
        lat FLOAT,
        long FLOAT,
        feature_class VARCHAR(1),
        feature_code VARCHAR(10),
        country_code VARCHAR(2),
        alt_country_code VARCHAR(60),
        admin_1_code VARCHAR(20),
        admin_2_code VARCHAR(20),
        admin_3_code VARCHAR(20),
        admin_4_code VARCHAR(20),
        population BIGINT,
        elevation INTEGER,
        dem INTEGER,
        timezone VARCHAR(40),
        modification_date VARCHAR(10)
    )
    """
)

con = None
con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')
try:
    cur = con.cursor()
    cur.execute(create_table_command)
    cur.close()
    con.commit()
    print('Successfully created table')
except (Exception, psycopg2.DatabaseError) as error:
    print(error)


# Code Scrap

The next 1 cell is an attempt to populate database by reading TSV file line by line.
Doesn't work, but leaving it here just for code syntax reference.

In [204]:
# insert_row = (
#     """
#     INSERT INTO cities (
#                 id, 
#                 name, 
#                 ascii, 
#                 alt_name, 
#                 lat, 
#                 long, 
#                 feature_class,
#                 feature_code, 
#                 country_code,
#                 alt_country_code,
#                 admin_1_code,
#                 admin_2_code,
#                 admin_3_code,
#                 admin_4_code,
#                 population,
#                 elevation,
#                 dem,
#                 timezone,
#                 modification_date) 
#         VALUES (
#             %s, %s, %s, %s, %s,
#             %s, %s, %s, %s, %s,
#             %s, %s, %s, %s, %s,
#             %s, %s, %s, %s);
#     """
# )

# insert_values = (
# '''
#     INSERT INTO cities VALUES (
#             %s, %s, %s, %s, %s,
#             %s, %s, %s, %s, %s,
#             %s, %s, %s, %s, %s,
#             %s, %s, %s, %s);
# ''')



# con = None
# con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')

# with open('new.csv','r') as csvin:
#     csvin = csv.reader(csvin, delimiter=',')
#     for index, row in enumerate(csvin):
#         if index > 1:
#             try:
#                 cur = con.cursor()
#                 cur.execute(insert_row,
#                             (row[0],
#                             row[1],
#                             row[2],
#                             row[3],
#                             row[4] or None,
#                             row[5] or None,
#                             row[6],
#                             row[7],
#                             row[8],
#                             row[9] or None,
#                             row[10] or None,
#                             row[11] or None,
#                             row[12],
#                             row[13],
#                             population,
#                             elevation,
#                             dem,
#                             row[17],
#                             row[18],))
#             except (Exception, psycopg2.DatabaseError) as error:
#                 print(f'Error: {error}')
#     cur.close()
#     con.commit()
#     con.close()



Error: relation "cities" does not exist
LINE 2:     INSERT INTO cities (
                        ^

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current trans

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is ab

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is ab

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is ab

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is ab

...

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is ab

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is ab

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is ab

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is ab

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is aborted, commands ignored until end of transaction block

Error: current transaction is ab

In [216]:
# Populate database from CSV File
con = None
con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')
cur = con.cursor()

with open('cities_canada-usa.tsv','r',encoding='UTF-8') as tsvin:
    next(tsvin)
    cur.copy_from(tsvin, 'cities', sep='\t', null='')
    
con.commit()

In [205]:
# Delete table cities
con = None
con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')
cur = con.cursor()

try:
    cur.execute('DELETE FROM cities')
except (Exception, psycopg2.DatabaseError) as error:
    print(f'Error: {error}')
finally:
    con.commit()
    cur.close()
    con.close()

Error: relation "cities" does not exist
LINE 1: DELETE FROM cities
                    ^



In [285]:
# Create class of data objects

class City:
    def __init__(self, name, lat, long):
        self.name = name
        self.lat = lat
        self.long = long
        self.confidence_level = 0.0
        self.edit_distance = 0
        self.physical_distance = None
        
#     @property
#     def confidence_level(self): # implements the get - this name is *the* name
#         return self._confidence_level

#     @confidence_level.setter
#     def confidence_level(self, value): # name unimportant, except for debugging
#         self._confidence_level = value

#     @confidence_level.deleter
#     def confidence_level(self): # again, name unimportant except for debugging
#         del self._confidence_level
        
#     @property
#     def edit_distance(self): # implements the get - this name is *the* name
#         return self._edit_distance

#     @edit_distance.setter
#     def edit_distance(self, value): # name unimportant, except for debugging
#         self._edit_distance = value

#     @edit_distance.deleter
#     def confidence_level(self): # again, name unimportant except for debugging
#         del self._edit_distance
        
#     @property
#     def physical_distance(self): # implements the get - this name is *the* name
#         return self._physical_distance

#     @physical_distance.setter
#     def physical_distance(self, value): # name unimportant, except for debugging
#         self._physical_distance = value

#     @physical_distance.deleter
#     def physical_distance(self): # again, name unimportant except for debugging
#         del self._physical_distance
        
    def __repr__(self):
        return f'{self.name}  (Confidence {self.confidence_level}) (Edit Distance {self.edit_distance}) (Distance {self.physical_distance})'

In [290]:
# Read all values from postgres database

def search(query, lat=None, long=None):
    con = None
    con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')
    cur = con.cursor()
    cur.execute('SELECT * FROM cities')
    rows = cur.fetchall()
    
    similar_cities = []
    for row in rows:
        confidence_graph = edge_d(query, row[1])
        if confidence_graph[-1][-1] < 3:
            city = City(row[1], row[4], row[5])
            city.edit_distance = confidence_graph[-1][-1]
            if lat and long:
                distance = lat_long_d(lat, long, row[4], row[5])
                city.physical_distance = distance
            
            # Calculate confidence level
            confidence_level = 0.0
            if city.physical_distance:
                confidence_level = 0.8 * (1 / (1 + city.edit_distance)) + 0.2 * ( 1 / (1 + city.physical_distance / 100))
            else:
                confidence_level = (1 / (1 + city.edit_distance))
            city.confidence_level = confidence_level
            similar_cities.append(city)
            

    cur.close()
    con.close()
    return similar_cities
    
search('London')

[London  (Confidence 1.0) (Edit Distance 0) (Distance 0.0),
 Linton  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 London  (Confidence 1.0) (Edit Distance 0) (Distance 0.0),
 Lyndon  (Confidence 0.5) (Edit Distance 1) (Distance 0.0),
 Landen  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 London  (Confidence 1.0) (Edit Distance 0) (Distance 0.0),
 Loudon  (Confidence 0.5) (Edit Distance 1) (Distance 0.0),
 Hondo  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 Rendon  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 Lorton  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 Mendon  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 Monson  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 Linden  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 Lyndon  (Confidence 0.5) (Edit Distance 1) (Distance 0.0),
 Lindon  (Confidence 0.5) (Edit Distance 

In [289]:
results = search('Londo')
sorted_search = sorted(results, key=lambda x: x.edit_distance)
sorted_search

[London  (Confidence 0.5) (Edit Distance 1) (Distance 0.0),
 London  (Confidence 0.5) (Edit Distance 1) (Distance 0.0),
 London  (Confidence 0.5) (Edit Distance 1) (Distance 0.0),
 Hondo  (Confidence 0.5) (Edit Distance 1) (Distance 0.0),
 Lyndon  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 Loudon  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 Lodi  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 Lyndon  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 Linda  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 Lodi  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0),
 Lindon  (Confidence 0.3333333333333333) (Edit Distance 2) (Distance 0.0)]

In [244]:
def lat_long_d(_lat_a, _long_a, _lat_b, _long_b):
    '''
    Returns distance between two points in km
    '''
    lat_a = math.radians(_lat_a)
    lat_b = math.radians(_lat_b)
    long_a = math.radians(_long_a)
    long_b = math.radians(_long_b)
    
    # approximate radius of earth in km
    R = 6373.0
    dlon = long_b - long_a
    dlat = lat_b - lat_at

    a = math.sin(dlat / 2)**2 + math.cos(lat_a) * math.cos(lat_b) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance
lat_long_d(45.5017089, -73.5728828, 43.4821971, -80.6167601)

602.0879382136474